In [5]:
import os, sys
# 1) Clone repo
if not os.path.exists('/content/bdh_challenge_2025'):
    %cd /content
    !git clone https://github.com/arionandrei2000/bdh_challenge_2025.git

# 2) Move into repo root
%cd /content/bdh_challenge_2025
print("PWD:", os.getcwd())
print("Repo contents:", os.listdir())

# 3) src is under notebooks/src
src_parent = "/content/bdh_challenge_2025/notebooks"
if src_parent not in sys.path:
    sys.path.append(src_parent)
print("sys.path updated; ready to import src.")

/content/bdh_challenge_2025
PWD: /content/bdh_challenge_2025
Repo contents: ['gdc-client', 'notebooks', 'README.md', 'gdc.zip', '.git']
sys.path updated; ready to import src.


In [6]:
# - pandas / numpy: data manipulation
# - pyarrow: for saving Parquet
# - requests: HTTP requests to GDC API
# - tqdm: progress bars
# - pyranges: may be useful later for genomic ranges
!pip install pandas numpy pyarrow requests tqdm pyranges --quiet

# Download GDC client
!wget https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Ubuntu_x64.zip -O gdc.zip -q
!unzip -o gdc.zip > /dev/null
!chmod +x gdc-client
print("Installed Python deps and downloaded gdc-client.")

^C
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of gdc.zip or
        gdc.zip.zip, and cannot find gdc.zip.ZIP, period.
Installed Python deps and downloaded gdc-client.


In [7]:
# 02) Mount GDrive
# This lets us store large TCGA data in Drive so it persists across sessions
from google.colab import drive
drive.mount('/content/drive')
print(" Google Drive mounted at /content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Google Drive mounted at /content/drive


In [8]:
# 03) Project Paths in Google Dribe

from pathlib import Path
# Folder Google Drive where we'll store all TCGA data for this project.
PROJECT_ROOT = Path("/content/drive/MyDrive/bdh_challenge_2025_data")
PROJECT_ROOT.mkdir(exist_ok=True)

# Generic data directory
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Directory where raw TCGA STAR count files will be downloaded
RNA_DIR = PROJECT_ROOT / "tcga_rna"
RNA_DIR.mkdir(exist_ok=True)
print("Project root:", PROJECT_ROOT)
print("Data dir    :", DATA_DIR)
print("RNA dir     :", RNA_DIR)

Project root: /content/drive/MyDrive/bdh_challenge_2025_data
Data dir    : /content/drive/MyDrive/bdh_challenge_2025_data/data
RNA dir     : /content/drive/MyDrive/bdh_challenge_2025_data/tcga_rna


In [9]:
# 04) Imports and TCGA cohort
import os
import json
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

# We explicitly list the TCGA projects we will include. THIS CAN BE CHANGED!
BULK_RNABERT_PROJECTS = [
    "TCGA-BRCA",  # Breast invasive carcinoma
    "TCGA-BLCA",  # Bladder urothelial carcinoma
    "TCGA-GBM",   # Glioblastoma multiforme
    "TCGA-LGG",   # Lower grade glioma
    "TCGA-LUAD",  # Lung adenocarcinoma
    "TCGA-UCEC",  # Uterine corpus endometrial carcinoma
]

print("Cohorts for this run:")
for p in BULK_RNABERT_PROJECTS:
    print(" -", p)

Cohorts for this run:
 - TCGA-BRCA
 - TCGA-BLCA
 - TCGA-GBM
 - TCGA-LGG
 - TCGA-LUAD
 - TCGA-UCEC


In [10]:
# 05) QUERY GDC for gene expression (the Star Counts)

#  - data_category: Transcriptome Profiling
#  - data_type: Gene Expression Quantification
#  - projects: the 6 TCGA projects above

files_query = {
    "filters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": BULK_RNABERT_PROJECTS,
                },
            },
            {
                "op": "in",
                "content": {
                    "field": "data_category",
                    "value": ["Transcriptome Profiling"],
                },
            },
            {
                "op": "in",
                "content": {
                    "field": "data_type",
                    "value": ["Gene Expression Quantification"],
                },
            },
        ],
    },
    "format": "JSON",
    "size": 20000,
    "fields": (
        "file_id,file_name,"
        "cases.submitter_id,cases.project.project_id,"
        "data_category,data_type,analysis.workflow_type"
    ),
}

resp = requests.post(
    "https://api.gdc.cancer.gov/files",
    headers={"Content-Type": "application/json"},
    data=json.dumps(files_query),
)

resp_json = resp.json()
files_json = resp_json.get("data", {}).get("hits", [])

print("HTTP status:", resp.status_code)
print("Number of files for selected projects:", len(files_json))

if len(files_json) == 0:
    print(" GDC returned 0 files. Pagination/debug:")
    print(json.dumps(resp_json.get("data", {}).get("pagination", {}), indent=2))
    raise RuntimeError("GDC returned 0 files for this query. Adjust filters or check connection.")

# Convert JSON list to DataFrame
files_df = pd.json_normalize(files_json)

def _get_first_case(x):
    """Helper: for 'cases' field that is a list, return the first dict or {}."""
    if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict):
        return x[0]
    return {}

# Extract submitter_id and project_id from nested 'cases' field
files_df["submitter_id"] = files_df["cases"].apply(
    lambda x: _get_first_case(x).get("submitter_id", None)
)
files_df["project_id"] = files_df["cases"].apply(
    lambda x: _get_first_case(x).get("project_id", None)
)

print("Unique workflows in these files:")
print(files_df["analysis.workflow_type"].value_counts())

files_df[
    ["file_id", "file_name", "submitter_id", "project_id",
     "data_category", "data_type", "analysis.workflow_type"]
].head()

HTTP status: 200
Number of files for selected projects: 3777
Unique workflows in these files:
analysis.workflow_type
STAR - Counts    3777
Name: count, dtype: int64


,file_id,file_name,submitter_id,project_id,data_category,data_type,analysis.workflow_type
0,9dc09c86-c728-4bd9-b2b6-2d9962dad662,d1f1743c-5fd9-4ae8-90c2-8c3e2d475d1b.rna_seq.a...,TCGA-EW-A2FS,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
1,95668f0b-130d-44d4-94c0-ba7a4e7798e6,6365a756-2e65-42cb-be4f-1f726915ca94.rna_seq.a...,TCGA-OL-A6VR,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
2,461fda5d-d6e6-4354-b035-c302cc43b03f,30285113-d411-475a-a8ed-1fb66be72f28.rna_seq.a...,TCGA-E9-A226,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
3,30ff778c-844b-4140-9025-7ab1938f10a9,5167da8c-2b1c-4139-a2c4-355e9f07d0be.rna_seq.a...,TCGA-A8-A08H,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
4,427a04c9-9b48-49de-8a47-2adc4e1dd32a,fead73ce-2e66-4647-8b6c-b8bbdaaf30fe.rna_seq.a...,TCGA-D8-A27H,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts


In [11]:
# 06) Create MANIFEST & DOWNLOAD star counts TO GDrive

# GDC client expects a manifest file with an "id" column listing file_ids.
manifest = files_df[["file_id"]].rename(columns={"file_id": "id"})
manifest_path = PROJECT_ROOT / "manifest_star_counts.txt"
manifest.to_csv(manifest_path, sep="\t", index=False)

print("Manifest written to:", manifest_path)
print("Total files listed:", manifest.shape[0])

# Download all files into RNA_DIR (in Google Drive).
# gdc-client will skip files that are already complete.
!./gdc-client download -m {manifest_path} -d {RNA_DIR}

print(" Download finished (check tcga_rna/ in Google Drive).")

Manifest written to: /content/drive/MyDrive/bdh_challenge_2025_data/manifest_star_counts.txt
Total files listed: 3777
100% [#############################################] Time:  0:00:04 840.6 KiB/s 
ERROR: 5645320f-820e-45b7-b4cf-ea05085f53e3: Unable to connect to API: (('Connection aborted.', OSError("(104, 'ECONNRESET')"))). Is this url correct: 'https://api.gdc.cancer.gov/data/5645320f-820e-45b7-b4cf-ea05085f53e3'? Is there a connection to the API? Is the server running?
ERROR: 74c60b29-cbf2-4398-aae0-372561a98682: Unable to connect to API: (('Connection aborted.', OSError("(104, 'ECONNRESET')"))). Is this url correct: 'https://api.gdc.cancer.gov/data/74c60b29-cbf2-4398-aae0-372561a98682'? Is there a connection to the API? Is the server running?
ERROR: 03adfb2d-63a4-43fb-87ae-0d174116490e: Unable to connect to API: (('Connection aborted.', OSError("(104, 'ECONNRESET')"))). Is this url correct: 'https://api.gdc.cancer.gov/data/03adfb2d-63a4-43fb-87ae-0d174116490e'? Is there a connect

In [12]:
# 07) Getting the STAR counts matrix

# This version:
#  - Only processes STAR gene count files (augmented_star_gene_counts.tsv)
#  - Reads with header row
#  - Drops N_* and __* summary rows
#  - Works with partial downloads
#  - Stores counts as float32

id_to_sample = dict(zip(files_df["file_id"], files_df["submitter_id"]))
print("Number of file_id → sample mappings:", len(id_to_sample))

all_files = []
for root, dirs, files in os.walk(RNA_DIR):
    for f in files:
        # STAR gene count files usually contain 'rna_seq' and 'gene_counts'
        if f.endswith(".tsv") and "rna_seq" in f and "gene_counts" in f:
            all_files.append(os.path.join(root, f))

print("Number of STAR gene count files on disk:", len(all_files))

# OPTIONAL: limit for debugging
MAX_FILES = None
if MAX_FILES is not None:
    all_files = all_files[:MAX_FILES]
    print(f"Using only first {len(all_files)} files.")

gene_index = None
matrix = {}
skipped = []

for fpath in tqdm(all_files):
    fname = os.path.basename(fpath)

    # Folder name created by gdc-client is the file_id
    file_id = os.path.basename(os.path.dirname(fpath))
    sample = id_to_sample.get(file_id)
    if sample is None:
        skipped.append((fname, "no sample mapping"))
        continue

    # Read gene counts table; ignore comment lines starting with '#'
    df = pd.read_csv(fpath, sep="\t", comment="#")

    # Identify gene ID column
    if "gene_id" in df.columns:
        gene_col = "gene_id"
    elif "Geneid" in df.columns:
        gene_col = "Geneid"
    else:
        skipped.append((fname, f"no gene_id column, cols = {df.columns.tolist()}"))
        continue

    # Identify counts column
    if "unstranded" in df.columns:
        counts_col = "unstranded"
    else:
        # pick the first non-gene_id/gene_name column as fallback
        candidate_cols = [c for c in df.columns if c not in [gene_col, "gene_name"]]
        if not candidate_cols:
            skipped.append((fname, "no suitable count column"))
            continue
        counts_col = candidate_cols[0]

    # Drop technical summary rows: N_unmapped, N_multimapping, __no_feature, etc.
    df = df[~df[gene_col].astype(str).str.startswith("N_")]
    df = df[~df[gene_col].astype(str).str.startswith("__")]
    df = df.reset_index(drop=True)

    # Initialize or check gene ordering
    if gene_index is None:
        gene_index = df[gene_col].values
    else:
        if not np.array_equal(gene_index, df[gene_col].values):
            raise ValueError(f"Gene order mismatch in file: {fpath}")

    # Extract counts as float32 (saves RAM vs float64)
    counts = pd.to_numeric(df[counts_col], errors="raise").astype("float32").values
    matrix[sample] = counts

# Build counts matrix (genes x samples) and store as float32
expr_counts = pd.DataFrame(matrix, index=gene_index).astype("float32")

print("Counts matrix shape (genes x samples):", expr_counts.shape)
display(expr_counts.iloc[:5, :5])

print("Skipped files:", len(skipped))
if skipped:
    print(skipped[:5])

Number of file_id → sample mappings: 3777
Number of STAR gene count files on disk: 3777


100%|██████████| 3777/3777 [09:00<00:00,  6.99it/s]


Counts matrix shape (genes x samples): (60660, 3385)


,TCGA-12-0827,TCGA-P5-A5EY,TCGA-AR-A1AW,TCGA-E9-A1N5,TCGA-97-7941
ENSG00000000003.15,2371.0,2377.0,2273.0,2036.0,11528.0
ENSG00000000005.6,37.0,8.0,13.0,39.0,13.0
ENSG00000000419.13,579.0,649.0,2077.0,703.0,1547.0
ENSG00000000457.14,853.0,331.0,1459.0,764.0,1257.0
ENSG00000000460.17,519.0,79.0,726.0,251.0,232.0


Skipped files: 0


In [13]:
# 08) Doing TPM-like normalization on Counts
# We approximatd TPM by:
#   TPM_like = (counts / sum(counts)) * 1e6 per sample.
# This keeps per-sample library size comparable across samples.

# Very close to using FPKM→TPM for a lot of downstream modeling use cases. LET'S double check this! Why?

def counts_to_tpm_like(column: pd.Series) -> pd.Series:
    total = column.sum()
    if total == 0:
        return column
    return (column / total) * 1e6

expr_tpm = expr_counts.apply(counts_to_tpm_like, axis=0)
print("TPM-like matrix shape (genes x samples):", expr_tpm.shape)
expr_tpm.iloc[:5, :5]


TPM-like matrix shape (genes x samples): (60660, 3385)


,TCGA-12-0827,TCGA-P5-A5EY,TCGA-AR-A1AW,TCGA-E9-A1N5,TCGA-97-7941
ENSG00000000003.15,51.796654,47.031635,39.358894,69.550377,178.855652
ENSG00000000005.6,0.808299,0.158289,0.225106,1.332252,0.201694
ENSG00000000419.13,12.648783,12.841199,35.964989,24.014692,24.001535
ENSG00000000457.14,18.634562,6.549210,25.263803,26.098471,19.502218
ENSG00000000460.17,11.338028,1.563104,12.571296,8.574236,3.599455


In [14]:
# 09) Clean genes, reorient to samples x genes, log-transform, and save

# Starting point:
#   - expr_counts : (genes x samples), float32
#   - expr_tpm    : (genes x samples), TPM-like
#
# This cell:
#   (1) Strips Ensembl version suffixes
#   (2) Removes duplicates in BOTH matrices
#   (3) Intersects gene sets so counts & TPM have identical genes
#   (4) Reorients to (samples x genes)
#   (5) Applies log10(1 + TPM-like)
#   (6) Saves outputs

import gc
import numpy as np
import os

# 1) Strip Ensembl version suffix from gene IDs
expr_tpm.index    = expr_tpm.index.to_series().str.split(".").str[0]
expr_counts.index = expr_counts.index.to_series().str.split(".").str[0]

# 2) Remove duplicate genes IN BOTH MATRICES
expr_tpm    = expr_tpm[~expr_tpm.index.duplicated(keep="first")]
expr_counts = expr_counts[~expr_counts.index.duplicated(keep="first")]

# 3) Intersect genes so both matrices have identical gene sets
common_genes = expr_tpm.index.intersection(expr_counts.index)
expr_tpm    = expr_tpm.loc[common_genes].sort_index()
expr_counts = expr_counts.loc[common_genes].sort_index()

print("After strip + dedupe + intersect:")
print("  expr_counts shape:", expr_counts.shape)
print("  expr_tpm    shape:", expr_tpm.shape)

# 4) Reorient to (samples x genes)
expr_counts_sxg = expr_counts.T.astype("float32").copy()
expr_tpm_sxg    = expr_tpm.T.astype("float32").copy()

expr_counts_sxg.index.name = "sample_id"
expr_tpm_sxg.index.name    = "sample_id"

print("Counts   (samples x genes):", expr_counts_sxg.shape)
print("TPM-like (samples x genes):", expr_tpm_sxg.shape)

gc.collect()

# 5) Log-transform: log10(1 + TPM-like)

expr_tpm_log = np.log10(1.0 + expr_tpm_sxg.astype("float32"))
print("Log10(1 + TPM-like) shape:", expr_tpm_log.shape)

# 6) Save matrices to disk

if "BASE_DIR" in globals():
    OUT_DIR = os.path.join(BASE_DIR, "processed")
else:
    OUT_DIR = os.path.join(os.getcwd(), "processed")

os.makedirs(OUT_DIR, exist_ok=True)

expr_counts_sxg.to_parquet(os.path.join(OUT_DIR, "tcga_star_counts_sxg.parquet"))
expr_tpm_sxg.to_parquet(os.path.join(OUT_DIR, "tcga_tpm_like_sxg.parquet"))
expr_tpm_log.to_parquet(os.path.join(OUT_DIR, "tcga_tpm_like_log10_sxg.parquet"))

print(" Saved processed matrices to:", OUT_DIR)

# Optional: verify no duplicate gene columns remain
dups = expr_counts_sxg.columns[expr_counts_sxg.columns.duplicated()]
print("Duplicate gene columns in counts_sxg:", dups.nunique())


After strip + dedupe + intersect:
  expr_counts shape: (60616, 3385)
  expr_tpm    shape: (60616, 3385)
Counts   (samples x genes): (3385, 60616)
TPM-like (samples x genes): (3385, 60616)
Log10(1 + TPM-like) shape: (3385, 60616)
 Saved processed matrices to: /content/bdh_challenge_2025/processed
Duplicate gene columns in counts_sxg: 0
